In [1]:
from typing import AnyStr
import os
os.chdir("..")
os.chdir("src")
from data_cleaning import DataCleaning
os.chdir("..")
from transformers import pipeline
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
pd.set_option('display.max_colwidth', None)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

pipe = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7",
    framework="pt",
    device=device,
    batch_size=16,
)

2024-04-13 21:48:32.452490: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 21:48:32.454727: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 21:48:32.482546: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 21:48:32.979012: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [2]:
data_cleaning_pipeline = DataCleaning()
df = data_cleaning_pipeline.run()

2024-04-13 21:49:32 - INFO - Data loaded!
2024-04-13 21:49:32 - INFO - Null values cleaned!
2024-04-13 21:49:32 - INFO - Data types asserted!
2024-04-13 21:49:32 - INFO - Full data cleaned!


In [3]:
df_trial = df.sample(10000)

In [10]:
dataset = Dataset.from_pandas(df_trial[["DS_OBJETO"]])

In [15]:
candidate_labels = ["veículos", "construção", "saúde", "educação", "segurança pública"]
hypothesis_template = "Esse texto é uma licitação pública, ou seja, a aquisição de determinado bem ou serviço por um órgão público. O tópico desse exemplo é {}."
dataset = Dataset.from_pandas(df_trial[["DS_OBJETO"]])
output_total = pipe(sequences = dataset["DS_OBJETO"],
                     candidate_labels=candidate_labels,
                     hypothesis_template=hypothesis_template,                    
                     multi_label=False)
output_total = [output["labels"][0] for output in output_total]

df_trial["label"] = output_total

In [16]:
df_trial["label"].value_counts()
# df_trial[["DS_OBJETO", "label"]].to_csv("output_llm_10000.csv", index=False)

label
segurança pública    3145
saúde                3057
construção           1572
educação             1120
veículos             1106
Name: count, dtype: int64

In [21]:
df_trial[df_trial["label"]=="veículos"][["DS_OBJETO", "label"]]

,DS_OBJETO,label
59656,"Locação de seis (6) caminhões-pipa, conforme especificações constantes no Termo de Referência.",veículos
106780,"AQUISIÇÃO DE PEÇAS PARA MANUTENÇÃO DE MÁQUINAS, PELO PERÍODO DE 12 (DOZE) MESES.",veículos
124208,"Pregão Eletrônico 36/2021 realizado para aquisição de Pneus para atender as necessidades das SEC DE EDUCAÇÃO e SEC DE AGRICULTURA, conforme protocolos nº1703 e 1597/2021.",veículos
116769,REGISTRO DE PREÇOS PARA EXECUÇÃO DO SERVIÇO DE CAMINHÃO-PIPA COMMOTORISTA/OPERADOR E AUXILIAR PARA SUPRIMENTO DE ÁGUA – SURPLA - SUPERINTENDÊNCIAREGIONAL PLANALTO.,veículos
120724,AQUISIÇÃO DE UMA RETROESCAVADEIRA.,veículos
...,...,...
129700,"CREDENCIAMENTO para prestação de serviços de PUBLICIDADE em JORNAIS, REVISTAS, MATERIAL INFORMATIVO e MIDIA DIGITAL, com CIRCULAÇÃO no Município de ILÓPOLIS/RS e Região do Vale do Taquari, para veiculação de publicidade institucional do Poder Público Municipal de Ilópolis/RS, através de publicações de atos legais, programas de utilidade pública de caráter informativo, educativo e de orientação social, através da imprensa escrita e online",veículos
36608,"Contratação de empresa especializada para execução e instalação de abrigos de ônibus na cidade de Passo Fundo/RS, com o fornecimento dos materiais e da mão de obra necessários, conforme especificações, quantidades e condições constantes no edital e seus anexos.",veículos
8613,"CONTRATAÇÃO DE EMPRESA ESPECIALIZADA PARA PRESTAÇÃO DE SERVIÇO DE LOCAÇÃO DE 03 (TRÊS) ÔNIBUS, COMPORTANDO MOTORISTA E COMBUSTÍVEL, PARA VIAGENS AS CIDADES DE NOVO HAMBURGO/RS, AURORA/SC E ESTEIO/RS, CONFORME TERMOS DE REFERÊNCIA ENCAMINHADOS PELAS SECRETARIAS MUNICIPAIS DE AGRICULTURA E PESCA E DE EDUCAÇÃO E CULTURA, ATRAVÉS DOS MEMORANDOS N° 4.603/2023/SMAP e N° 4.807/2023/SMEC. (P.A. N° 122/2023 - P.E. N° 012/2023)",veículos
49158,"aquisição de peças e serviço de mão de obra para conserto do motor do Caminhão Agrale 13000, placas ISB 6337, do motor da Van Fiat Ducato, Placas IXG 1586 e do Britador Móvel modelo 62x40",veículos


In [35]:
pipe = pipeline(model="facebook/bart-large-mnli", device=device)

dataset = Dataset.from_pandas(df_trial[["DS_OBJETO"]])
output_total = pipe(dataset["DS_OBJETO"], zeroshot_topic_list, multi_label=False)
output_total = [output["labels"][0] for output in output_total]

In [12]:
df_trial = pd.read_csv("/home/bruno/mestrado-ufrgs/mestrado-ufrgs-cmp617-tce/models/llm/output_llm_10000.csv")

In [23]:
df_trial[df_trial["label"]=="education"][["DS_OBJETO", "label"]]

,DS_OBJETO,label
17810,Registro de Preços para contratação de empresa para realização de esterilização e inserção de microchip em animais domésticos (cães fêmeas).,education
113633,"PESQUISA DE PREÇO PARA LICITAÇÃO DE MATERIAL ELÉTRICO, CONFORME PROTOCOLO N° 3009/2021",education
33205,"Contratação de empresa para elaboração do PGR, PCMSO, LTCAT e LTIP.",education
24710,"CONTRATAÇÃO DE EMPRESA PARA FORNECIMENTO DE SOFTWARE DE GESTÃO PARA A SECRETARIA MUNICIPAL DE EDUCAÇÃO, CONFORME SOLICITAÇÃO DA COORDENADORIA DE TECNOLOGIA DE INFORMAÇÃO E COMUNICAÇÃO",education
101538,"CONTRATAÇÃO DE EMPRESA ESPECIALIZADA PARA EXECUÇÃO DE PLANO DE PREVENÇÃO E PROTEÇÃO CONTRA INCENDIO (PPCI) NA EMEF CARLOS GOMES, LOCALIZADA EM VILA CASCATA, CONFORME PROJETOS EM ANEXO.",education
...,...,...
24178,"AQUISIÇÃO DE MERENDA ESCOLAR PARA AS ESCOLAS, E.M.E.I. SONHO MEU E PROJETO CONTRA TURNO ESCOLAR PARA O 2° SEMESTRE DE 2022.",education
59021,"Contratação de Empresa para a prestação de serviços de Transporte Escolar para alunos matriculados na Pré-Escola, da Educação Infantil, nas Séries Iniciais e Finais do Ensino Fundamental e no Ensino Médio, residentes no Município de Maratá.",education
27465,Aquisição de gêneros alimentícios para distribuição nas escolas da rede municipal de ensino,education
93694,Contratação de Prestação de Serviços de Transporte Escolar,education


In [27]:
# Nome do modelo que você está usando
model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"

# Criar o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Criar o modelo
model = AutoModel.from_pretrained(model_name)


2024-04-05 22:59:12.626741: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-05 22:59:14.009266: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-05 22:59:15.373602: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [56]:
docs = df_trial["DS_OBJETO"].tolist()

# Tokenizar os documentos
tokenized_corpus = tokenizer(docs, padding=True, truncation=True, return_tensors="pt")

# Extrair os tokens
tokens_tensor = tokenized_corpus["input_ids"]

vec = []
for tokens in tokens_tensor:
    # Remover os zeros e converter os IDs para palavras usando o tokenizer
    tokens = [token for token in tokens.numpy().tolist() if token != 0]
    tokens_text = tokenizer.convert_ids_to_tokens(tokens)
    vec.append(tokens_text)

id2word = Dictionary(vec)

corpus = [id2word.doc2bow(text) for text in vec]

## talvez precise ajustar a qusetao dos tokens

In [72]:
vec

[['[CLS]',
  '▁Aqui',
  'sição',
  '▁de',
  '▁bloc',
  'os',
  '▁de',
  '▁pedr',
  'a',
  '▁gr',
  'ês',
  ',',
  '▁para',
  '▁rede',
  '▁de',
  '▁dren',
  'agem',
  '▁plu',
  'vial',
  '▁do',
  '▁Munic',
  'ípi',
  'o',
  '.',
  '[SEP]'],
 ['[CLS]',
  '▁Edit',
  'al',
  '▁de',
  '▁preg',
  'ão',
  '▁exclusiv',
  'o',
  '▁',
  'à',
  '▁participa',
  'ção',
  '▁de',
  '▁micro',
  'empresa',
  '▁',
  'e',
  '▁empresa',
  '▁de',
  '▁',
  'pequen',
  'o',
  '▁porte',
  '▁para',
  '▁presta',
  'ção',
  '▁de',
  '▁',
  's',
  'erviços',
  '▁de',
  '▁E',
  'labora',
  'ção',
  '▁de',
  '▁P',
  'PCI',
  '▁(',
  'Plan',
  'o',
  '▁de',
  '▁Preven',
  'ção',
  '▁',
  'e',
  '▁Prote',
  'ção',
  '▁Contra',
  '▁Inc',
  'ên',
  'dio',
  ').',
  '[SEP]'],
 ['[CLS]',
  '▁',
  'Registro',
  '▁de',
  '▁',
  'Preço',
  's',
  '▁para',
  '▁contrat',
  'ação',
  '▁de',
  '▁empresa',
  '▁para',
  '▁realiza',
  'ção',
  '▁de',
  '▁este',
  'ril',
  'ização',
  '▁',
  'e',
  '▁inser',
  'ção',
  '▁de',
  '▁m

In [70]:
coherence_model_lda = CoherenceModel(
    topics=df_trial["label"], texts=vec, dictionary=id2word, coherence="c_v", corpus=corpus
)
coherence_model_lda.get_coherence()

In [4]:
# precisa instalar o pytorch antes, pro meu caso foi o comando abaixo (mas no readme nao precisa passar o comando)
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 

## Embedder + Kmeans

In [ ]:
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import KMeans
# from sklearn.metrics.pairwise import cosine_distances
# import numpy as np

# embedder = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# data_cleaning_pipeline = DataCleaning()
# df = data_cleaning_pipeline.run()

# corpus = df["DS_OBJETO"].values.tolist()

# corpus_embeddings = embedder.encode(corpus)

# # Perform kmean clustering
# num_clusters = 7
# clustering_model = KMeans(n_clusters=num_clusters, random_state=42)
# clustering_model.fit(corpus_embeddings)
# cluster_assignment = clustering_model.labels_

# np.mean(cosine_distances(corpus_embeddings, clustering_model.cluster_centers_))